In [167]:
data = "/Users/jon.clucas/probcomp/train_test_data/unsplit_no_totals.csv"

In [213]:
import numpy as np
import pandas as pd
import phat


def vector_vector(df):
    d = list(list(df.loc[df[c]].index.values) for c in df.columns)
    return(d)


def vector_vector_T(df):
    df_T = df.T
    df_T.reset_index(inplace=True)
    d = list(list(df_T.loc[df_T[c]].index.values) for c in [c for c in df_T.columns if c != "index"])
    return(d)

In [169]:
df = pd.read_csv(data)

In [170]:
dxes = {("Anx", "adhd", "asd"):
           {(0,0,0),
           (1,0,0),
           (0,1,0),
           (0,0,1),
           (1,1,0),
           (1,0,1),
           (0,1,1),
           (1,1,1)}
       }

In [171]:
subdfs = {}
for k in dxes:
    for r in dxes[k]:
        d = set(zip(k, r))
        subdfs[str(dict(d))] = df.loc[
            eval(
                " & ".join(
                    {
                        "(df[\"{0}\"] == {1})".format(key, value) for key, value in d
                    }
                )
            )
        ].copy()

In [172]:
for n in subdfs:
    print(n, ":", len(subdfs[n]))

{'adhd': 1, 'asd': 0, 'Anx': 1} : 19
{'asd': 1, 'adhd': 1, 'Anx': 0} : 12
{'asd': 1, 'adhd': 1, 'Anx': 1} : 13
{'asd': 0, 'Anx': 1, 'adhd': 0} : 17
{'asd': 1, 'Anx': 0, 'adhd': 0} : 6
{'asd': 1, 'Anx': 1, 'adhd': 0} : 3
{'Anx': 0, 'asd': 0, 'adhd': 0} : 91
{'adhd': 1, 'Anx': 0, 'asd': 0} : 60


In [215]:
for n in subdfs:
    df_in_use = subdfs[n].loc[subdfs[n]["train"]].drop(
        ["Dx", "Anx", "adhd", "asd", "train", "Sex", "Age", "EID"],
        axis=1
    ).copy()
    df_in_use.reset_index(inplace=True, drop=True)
    for col in [col for col in df_in_use.columns if col != "EID"]:
        df_in_use[col] = (df_in_use[col] == max(df_in_use[col]))
    boundary_matrix = phat.boundary_matrix(representation = phat.representations.vector_vector)

    boundary_matrix.source = vector_vector_T(df_in_use)
    
    print("The boundary matrix has %d columns:" % len(boundary_matrix.columns))
    for col in boundary_matrix.columns:
        s = "Column %d represents a cell of dimension %d." % (col.index, col.dimension)
        if (col.boundary):
            s = s + " Its boundary consists of the cells " + " ".join([str(c) for c in col.boundary])
        print(s)
    print("Overall, the boundary matrix has %d entries." % len(boundary_matrix))

    pairs = boundary_matrix.compute_persistence_pairs()

    pairs.sort()

    print()
    print("There are %d persistence pairs: " % len(pairs))
    for pair in pairs:
        print("Birth: %d, Death: %d" % pair)


The boundary matrix has 0 columns:
Overall, the boundary matrix has 0 entries.

There are 0 persistence pairs: 
The boundary matrix has 0 columns:
Overall, the boundary matrix has 0 entries.

There are 0 persistence pairs: 
The boundary matrix has 0 columns:
Overall, the boundary matrix has 0 entries.

There are 0 persistence pairs: 
The boundary matrix has 0 columns:
Overall, the boundary matrix has 0 entries.

There are 0 persistence pairs: 
The boundary matrix has 0 columns:
Overall, the boundary matrix has 0 entries.

There are 0 persistence pairs: 
The boundary matrix has 0 columns:
Overall, the boundary matrix has 0 entries.

There are 0 persistence pairs: 
The boundary matrix has 0 columns:
Overall, the boundary matrix has 0 entries.

There are 0 persistence pairs: 
The boundary matrix has 0 columns:
Overall, the boundary matrix has 0 entries.

There are 0 persistence pairs: 
